In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Sample dataset (replace with your own dataset)
# Assuming features X and labels y
X = np.random.rand(1000, 10)  # Sample features
y = np.random.randint(2, size=1000)  # Sample labels (0: legitimate, 1: fraudulent)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [2]:
X_test_scaled

array([[ 1.34097909,  0.53665035, -0.03969746, ..., -1.58790187,
         1.27886282,  0.01703712],
       [-1.46361144,  0.6498567 , -1.00003221, ..., -0.65446889,
        -1.25772214, -1.33181661],
       [ 1.71293858,  0.68255178, -0.76028225, ...,  0.02352631,
         0.2746431 , -0.99738028],
       ...,
       [ 0.52705962,  0.55533654, -0.05997995, ..., -1.03896482,
         1.12166898,  0.5147828 ],
       [-0.60041192,  1.28580618,  0.02047906, ...,  1.49285753,
        -0.15198395, -1.47674454],
       [ 1.2830852 , -1.16471454,  0.37075733, ...,  0.05938686,
        -0.77278786,  1.63710599]])

In [4]:
# Variational Autoencoder (VAE) for feature generation
latent_dim = 2  # Latent dimensionality
input_dim = X_train_scaled.shape[1]

# Encoder
inputs = Input(shape=(input_dim,))
h = Dense(64, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_h = Dense(64, activation='relu')
decoder_output = Dense(input_dim)
h_decoded = decoder_h(z)
outputs = decoder_output(h_decoded)

# Define VAE model
vae = Model(inputs, outputs)

# VAE loss
reconstruction_loss = mse(inputs, outputs)
kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)

# Compile VAE model
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train VAE
vae.fit(X_train_scaled, epochs=50, batch_size=32, validation_data=(X_test_scaled, None))

# Generate features using VAE
# encoded_features = vae.encoder.predict(X_train_scaled)  # Generated features

# Define encoder model to get encoded features
encoder = Model(inputs, z_mean)  # Assuming you want to use mean of the latent variables as features

# Generate features using VAE
encoded_features = encoder.predict(X_train_scaled)  # Generated features


# Reinforcement Learning Agent
class ReinforcementLearningAgent:
    def __init__(self, input_dim):
        self.input_dim = input_dim
        # Define your reinforcement learning model here

    def train(self, X, y):
        # Train your reinforcement learning model using generated features and labels
        pass

# Instantiate Reinforcement Learning Agent
agent = ReinforcementLearningAgent(input_dim=latent_dim)

# Train agent with generated features and labels
agent.train(encoded_features, y_train)


Epoch 1/50
25/25 [==============================] - 1s 20ms/step - loss: 1.1917 - val_loss: 1.0525
Epoch 2/50
25/25 [==============================] - 0s 6ms/step - loss: 1.0592 - val_loss: 1.0083
Epoch 3/50
25/25 [==============================] - 0s 6ms/step - loss: 1.0367 - val_loss: 0.9913
Epoch 4/50
25/25 [==============================] - 0s 6ms/step - loss: 1.0262 - val_loss: 0.9877
Epoch 5/50
25/25 [==============================] - 0s 6ms/step - loss: 1.0201 - val_loss: 0.9843
Epoch 6/50
25/25 [==============================] - 0s 7ms/step - loss: 1.0194 - val_loss: 0.9809
Epoch 7/50
25/25 [==============================] - 0s 7ms/step - loss: 1.0140 - val_loss: 0.9788
Epoch 8/50
25/25 [==============================] - 0s 7ms/step - loss: 1.0148 - val_loss: 0.9747
Epoch 9/50
25/25 [==============================] - 0s 7ms/step - loss: 1.0109 - val_loss: 0.9751
Epoch 10/50
25/25 [==============================] - 0s 7ms/step - loss: 1.0077 - val_loss: 0.9739
Epoch 11/50
25/25 

In [5]:
encoded_features

array([[ 0.02909293,  0.08323554],
       [-0.02986573,  0.01870459],
       [ 0.03549287,  0.01099828],
       ...,
       [ 0.01503094,  0.03040832],
       [ 0.05846537, -0.04461211],
       [ 0.01020505, -0.04779164]], dtype=float32)